In [1]:
# YOLO_PREDICTIONS -> [NN] -> VOT_GT

In [2]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
import os
import os.path as osp
import numpy as np

In [4]:
# GLOBALS
# vot_path = "../../../vot2016/"
vot_path = "/home/zabulskyy/Datasets/vot2016/"
# yolo_pred_path = "../yolo_predictions/extended"
yolo_pred_path = "/mnt/sshfs/YOLO-tracker/yolo_predictions/even_more"
crit_vals = {
    "iou": 1,
    "dist2": 1,
#     "abs_pos": 1,
#     "rel_pos": 1,
    "cc": 100,  # class_correspondence
}
with open("../../yoloTorch/data/coco.names", "r") as f:
    classes = f.read().split("\n")[:-1]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [5]:
def read_full_gt(vot_path=vot_path, force_square=True, return_format="dict"):
    vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
    if return_format == "list":
        X = list()
    else:
        X = dict()
    for vot_class in vot_classes:
        gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
        with open(gt_file, 'r') as file:
            l =file.read().split("\n")
            l.pop(-1)
            l = [[float(x) for x in y.split(",")] for y in l]
            if force_square:
                l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        
        if return_format == "list":
            X.append(l)
        else:
            X[vot_class] = l
                    
    return X

In [6]:
def read_all_yolo_preds(path=yolo_pred_path, pred_format="csv", return_format="dict"):
    if return_format == "list":
        X = list()
    else:
        X = dict()
    
    for vot_class in os.listdir(path):
        yolo_pred_file = osp.join(path, vot_class)
        with open(yolo_pred_file, 'r') as file:
            l = file.read().split("\n")
            l.pop(-1)
            l.pop(0)
            im_len = -1
            for i in l:
                if return_format == "list":
                    X.append([float(x) for x in i.split(",")])
                else:
                    X[vot_class[:-4]] = [float(x) for x in i.split(",")]  # [:-4] to remove the .csv or .txt format
                    
    return X

In [22]:
def read_all_gt(path=yolo_pred_path, pred_format="csv", return_format="dict"):
    if return_format == "list":
        X = list()
    else:
        X = dict()
    
    for vot_class in os.listdir(path):
        yolo_pred_file = osp.join(path, vot_class)
        with open(yolo_pred_file, 'r') as file:
            l = file.read().split("\n")
            l.pop(-1)
            l.pop(0)
            im_len = -1
            for i in l:
                if return_format == "list":
                    X.append([float(x) for x in i.split(",")])
                else:
                    X[vot_class[:-4]] = [float(x) for x in i.split(",")]  # [:-4] to remove the .csv or .txt format
                    
    return X

In [23]:
def read_csv(file):
    with open(file, 'r') as file:
        l = file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")[:-1]] for y in l]   
        return torch.tensor(l)
    

In [7]:
yp = read_all_yolo_preds()
yp

{'gymnastics4': [464.0,
  3.310213804244995,
  301.96319580078125,
  18.733238220214844,
  324.2966613769531,
  0.0010151999304071069,
  0.32297974824905396,
  73.0],
 'sphere': [200.0,
  380.6707458496094,
  165.94168090820312,
  450.49224853515625,
  221.63876342773438,
  0.0016972447047010064,
  0.8267019987106323,
  74.0],
 'gymnastics1': [566.0,
  199.6515655517578,
  90.91927337646484,
  202.5141143798828,
  94.1538314819336,
  0.0010955819161608815,
  0.2609655559062958,
  67.0],
 'shaking': [364.0,
  534.6719970703125,
  275.4270935058594,
  574.7601928710938,
  311.321533203125,
  0.0012959630694240332,
  0.672863781452179,
  74.0],
 'crossing': [130.0,
  280.0235900878906,
  284.3335876464844,
  289.401123046875,
  294.74822998046875,
  0.007909675128757954,
  0.9703932404518127,
  67.0],
 'bmx': [75.0,
  577.2583618164062,
  11.704617500305176,
  667.7908935546875,
  143.93777465820312,
  0.0019840355962514877,
  0.5709840655326843,
  74.0],
 'fernando': [291.0,
  2.03010940

In [26]:
X = yp

In [28]:
y = read_full_gt()

In [32]:
X["blanket"]

[224.0,
 137.0874481201172,
 121.8482437133789,
 161.82241821289062,
 136.82615661621094,
 0.0011430482845753431,
 0.15401728451251984,
 77.0]